# The Problem

Discussing things you care about can be difficult. The threat of abuse and harassment online means that many people stop expressing themselves and give up on seeking different opinions. Platforms struggle to effectively facilitate conversations, leading many communities to limit or completely shut down user comments.

In Toxic Comment Challenge competition held on Kaggle, the challenge was to build a multi-headed model that’s capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate.

*Credits: https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge*

# Import Library

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 53.2 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 895 kB 30.4 MB/s 
     |████████████████████████████████| 636 kB 57.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import tensorflow as tf
import logging
from tensorflow.keras.layers import (
    Dense,
    Flatten,
    Conv1D,
    Dropout,
    Input,
    GlobalAveragePooling1D
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras import regularizers
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import History, EarlyStopping, ModelCheckpoint
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re

# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast, TFAutoModel

# Check For TPU Availability

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.33.6.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.33.6.138:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Data Preprocessing

Do a simple data cleaning to lowercase text, split up contractions and replace some toxic words that were written together without spacing.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
def clean_text(text):
    text = text.lower()

    # Split up contractions
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')

    # Split up toxic words
    toxic_reference_dictionary = {"youfuck": "you fuck", 
                                  "fucksex": "fuck sex",
                                  "bitchbot": "bitch bot",
                                  "offfuck": "fuck off",
                                  "donkeysex": "donkey sex",
                                  "securityfuck": "security fuck",
                                  "ancestryfuck": "ancestry fuck",
                                  "turkeyfuck": "turkey fuck",
                                  "faggotgay": "faggot gay",
                                  "fuckbot": "fuck bot",
                                  "assfuckers": "ass fucker",
                                  "ckckck": "cock",
                                  "fuckfuck": "fuck",
                                  "lolol": "lol",
                                  "pussyfuck": "pussy fuck",
                                  "gaygay": "gay",
                                  "haha": "ha",
                                  "sucksuck": "suck"
                                  }
    for existing,new_word in toxic_reference_dictionary.items():
        text = text.replace(existing,new_word)
    return text

In [ ]:
train_df = pd.read_csv("./train.csv").fillna("blank")
test_df = pd.read_csv("./test.csv").fillna("blank")

In [ ]:
train_df['comment_text'] = train_df['comment_text'].map(lambda x : clean_text(x))
test_df['comment_text'] = test_df['comment_text'].map(lambda x : clean_text(x))

## Handling Class Imbalance

There are many approaches that we can apply but they generally fall within 2 major school-of-thoughts:
1. Sampling (ROSE, SMOTE, etc)
2. Class weighting

### Bigger problems to solve.....
While it is nice if we can try out all the techniques, it is unlikely that most of us have sufficient computation resources to do it. 

### Solution:
Due to limited computation time on Colab's free tier, I will only use the simplest method of sampling which is to balance out the number of clean and toxic comments relatively evenly. Surprisingly, this is sufficient to give us a reasonable baseline model!


|S/N:| Method | Description |
|--- | --- | --- |
|1| Simple Sampling | Sample 20,000 clean comments but extract **ALL** toxic comments. |
|2|ROSE| Naive strategy is to generate new samples by randomly sampling with replacement the current available samples
|3|SMOTE| Generate new samples in by interpolation
|4|Class Weights Adjustments (Manual)| Create a new column to classify between Toxic VS Clean comments and assign weights with formula: `(1/class) * (total/2.0)`|
|5|Class Weights Adjustments (Sklearn)| Use Sklearn class weight computation to automatically assign weights|


In [ ]:
# Split dataset into toxic and clean comments
LABEL_COLUMNS = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_toxic = train_df[train_df[LABEL_COLUMNS].sum(axis=1) > 0]
train_clean = train_df[train_df[LABEL_COLUMNS].sum(axis=1) == 0]

# Sampling toxic and clean comments dataset
final_training_df = pd.concat([
  train_toxic,
  train_clean.sample(20000)
])

# Shuffle dataset
final_training_df = final_training_df.sample(frac=1).reset_index(drop=True)

In [ ]:
# from sklearn.utils import class_weight

# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(train_df[LABEL_COLUMNS]),
#                                                  train_df[LABEL_COLUMNS])


## Encode Data & Labels

In [ ]:
# Name of the BERT model to use
model_name = 'bert-base-uncased'

# Max length of tokens
max_length = 512

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
#config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

def bert_encode(data):
    tokens = tokenizer.batch_encode_plus(
        data,
        add_special_tokens=True,
        max_length=max_length,
        truncation=True,
        padding=True, 
        return_tensors='tf',
        return_token_type_ids = False,
        return_attention_mask = True,
        verbose = True
    )
    return tf.constant(tokens["input_ids"])

# Split into training and validation dataset
training_dataset, val_dataset = train_test_split(final_training_df, test_size=0.1, random_state=42)

# Encode training and validation dataset - FEATURES
train_encoded = bert_encode(training_dataset.comment_text)
val_encoded = bert_encode(val_dataset.comment_text)

# Encode training and validation dataset - LABELS
train_labels=training_dataset[LABEL_COLUMNS].values
train_labels=train_labels.reshape(-1,len(LABEL_COLUMNS))

val_labels=val_dataset[LABEL_COLUMNS].values
val_labels=val_labels.reshape(-1,len(LABEL_COLUMNS))

print()
print(f'Train labels shape: {train_labels.shape}')

DEBUG:filelock:Attempting to acquire lock 139848561624976 on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
DEBUG:filelock:Lock 139848561624976 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139848561624976 on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
DEBUG:filelock:Lock 139848561624976 released on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
DEBUG:filelock:Attempting to acquire lock 139847743771856 on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Lock 139847743771856 acquired on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139847743771856 on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Lock 139847743771856 released on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Attempting to acquire lock 139847694924816 on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Lock 139847694924816 acquired on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139847694924816 on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Lock 139847694924816 released on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Attempting to acquire lock 139848559278160 on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
DEBUG:filelock:Lock 139848559278160 acquired on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139848559278160 on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
DEBUG:filelock:Lock 139848559278160 released on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


(32602, 6)


In [ ]:
# Fitting into a Tensor unit
train_dataset = (
    tf.data.Dataset.from_tensor_slices((train_encoded, train_labels))
    .shuffle(100)
    .batch(32)
).cache()

val_dataset = (
    tf.data.Dataset.from_tensor_slices((val_encoded, val_labels))
    .shuffle(100)
    .batch(32)
).cache()

# Create BERT Model

In [ ]:
def bert_tpu_model():
    
    # ========================================================
    # Results - 98.341% testing accuracy
    bert_encoder = TFAutoModel.from_pretrained('bert-base-uncased')

    input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')

    last_hidden_states = bert_encoder(input_ids)[0]

    clf_output = Flatten()(last_hidden_states)

    dense_01 = Dense(512, activation='relu', name='dense_01')(clf_output)
    dropout_01 = Dropout(0.5)(dense_01)

    dense_02 = Dense(512, activation='relu', name='dense_02')(dropout_01)
    dropout_02 = Dropout(0.5)(dense_02)

    out =Dense(6, activation='sigmoid', name='outputs')(dropout_02)

    model = Model(inputs=input_ids, outputs=out)

    return model

In [ ]:
with strategy.scope():
    model = bert_tpu_model()
    optimizer = Adam(learning_rate=1e-5, decay=1e-6)
    model.compile(loss='binary_crossentropy',
                    optimizer=optimizer,
                    metrics=['accuracy'])
    model.summary()

DEBUG:filelock:Attempting to acquire lock 139847654062736 on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock
DEBUG:filelock:Lock 139847654062736 acquired on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139847654062736 on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock
DEBUG:filelock:Lock 139847654062736 released on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification mo

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 512)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  TFBaseModelOutputWithPool 109482240 
_________________________________________________________________
flatten (Flatten)            (None, 393216)            0         
_________________________________________________________________
dense_01 (Dense)             (None, 512)               201327104 
_________________________________________________________________
dropout_37 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_02 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_38 (Dropout)         (None, 512)               0     

In [ ]:
history = History()

# Set early stopping
es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   verbose=1, 
                   patience=8)  # Training will wait 8 epochs to check for any improvement to validation accuracy

# Save best model to train epoch
checkpoint = ModelCheckpoint('./bert_sampling_wgts.hdf5', 
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=True, 
                             mode='max');  

callbacks = [es, checkpoint, history]

model.fit(
    train_dataset,
    batch_size=32,
    epochs=100,
    validation_data=val_dataset,
    verbose=1,
    callbacks=callbacks
)


Epoch 1/100


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 6) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 6) dtype=int64>]


1019/1019 [==============================] - ETA: 0s - loss: 0.2327 - accuracy: 0.5534

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 6) dtype=int64>]


1019/1019 [==============================] - 321s 230ms/step - loss: 0.2327 - accuracy: 0.5534 - val_loss: 0.1524 - val_accuracy: 0.7047

Epoch 00001: val_accuracy improved from -inf to 0.70466, saving model to /content/drive/MyDrive/Colab Notebooks/bert_sampling_wgts.hdf5
Epoch 2/100
1019/1019 [==============================] - 218s 214ms/step - loss: 0.1669 - accuracy: 0.5914 - val_loss: 0.1493 - val_accuracy: 0.7560

Epoch 00002: val_accuracy improved from 0.70466 to 0.75600, saving model to /content/drive/MyDrive/Colab Notebooks/bert_sampling_wgts.hdf5
Epoch 3/100
1019/1019 [==============================] - 218s 214ms/step - loss: 0.1490 - accuracy: 0.6270 - val_loss: 0.1374 - val_accuracy: 0.8366

Epoch 00003: val_accuracy improved from 0.75600 to 0.83660, saving model to /content/drive/MyDrive/Colab Notebooks/bert_sampling_wgts.hdf5
Epoch 4/100
1019/1019 [==============================] - 218s 214ms/step - loss: 0.1346 - accuracy: 0.6402 - val_loss: 0.1431 - val_accuracy: 0.9081

In [ ]:
# test_encoded = bert_encode(test_df.comment_text)

# # test_dataset = (
# #     tf.data.Dataset.from_tensor_slices((test_encoded))
# #     .shuffle(100)
# #     .batch(64)
# # ).cache()

In [ ]:
# predictions = model.predict(test_encoded)

In [ ]:
# final_df=pd.DataFrame(predictions,columns=list_classes)
# final_df['id'] = test_df['id']
# final_df=final_df[['id']+(list_classes)]
# final_df["comment_text"] = test_df['comment_text']
# final_df